# Titanic: Machine Learning from Disaster

In [3]:
# https://www.kaggle.com/c/titanic

In [1]:
import sklearn
import pandas as pd

In [2]:
# Загружаем данные из файлов
train = pd.read_csv('./titanic/train.csv')
test = pd.read_csv('./titanic/test.csv')

In [3]:
# сохраняем столбец идентификаторов пассажиров для тестовых данных
test_passenger_id = test['PassengerId']

In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Предобработка данных, добавление признаков

In [5]:
# https://www.kaggle.com/arthurtok/introduction-to-ensembling-stacking-in-python

import re

RARE_TITLES = ['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona']

# создаем словарь для исправления префиксов имени
TITLES = dict((title, 'Rare') for title in RARE_TITLES)

TITLES['Mlle'] = 'Miss'
TITLES['Ms'] = 'Miss'
TITLES['Mme'] = 'Mrs'

def get_title(name):
    """ Возвращает префикс имени """
    match = re.search(' ([A-Za-z]+)\.', name)
    if match:
        # исправляем опечатки в префиксе имени
        return TITLES.get(match.group(1), match.group(1))
    return ''

# Применяем процесс для обучающего и тестового наборов
for dataset in [test, train]:
    # длина имени
    dataset['Name_Length'] = train['Name'].apply(len)
    # была ли каюта у пассажира
    dataset['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
    # сколько членов семьи было на корабле
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    # флаг = 1, если путешествует в одиночестве
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
    # заполняем пропуски для Embarked
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
    # заполняем индексами
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    # заполняем пропуски Fare медианой
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
    # заполняем пропуски возраста медианой
    dataset['Age'] = dataset['Age'].fillna(train['Age'].median())
    # добавляем префикс имени как отдельный категориальный признак
    dataset['Title'] = dataset['Name'].apply(get_title)
    dataset['Title'] = dataset['Title'].map( {'Mr': 1, 
                                              'Miss': 2, 
                                              'Mrs': 3,
                                              'Master': 4,
                                              'Rare': 5} )
    # заполняем неизвестные префиксы нулями
    dataset['Title'] = dataset['Title'].fillna(0)
    # бинаризуем пол
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    # категоризируем стоимость билета
    dataset.loc[dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare'] = 2
    dataset.loc[dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    # категоризируем возраст
    dataset.loc[dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[dataset['Age'] > 64, 'Age']

In [48]:
# Удаляем неиспользуемые столбцы
DROP_COL = ['PassengerId', 'Name', 'Ticket', 'Cabin']

X_train = np.array(train.drop(DROP_COL + ['Survived'], axis=1))
y_train = np.array(train['Survived'])
X_test = np.array(test.drop(DROP_COL, axis=1))

## Предсказание моделей для стеккинга

In [106]:
import numpy as np
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

def cross_val_predict_proba(estimator, X_train, y_train, X_test, random_state=None, n_splits=5):
    y_test = np.zeros((len(X_test), n_splits), np.float32)
    
    kfold = KFold(n_splits=n_splits, 
                  shuffle=True,
                  random_state=random_state)

    y_predict = np.zeros_like(y_train, np.float32)
    for i, (train_idx, test_idx) in enumerate(kfold.split(y_train)):
        estimator.fit(X_train[train_idx], y_train[train_idx])
        y_predict[test_idx] = estimator.predict_proba(X_train[test_idx])[:, 1]
        y_test[:, i] = estimator.predict_proba(X_test)[:, 1]
    
    return y_predict, np.mean(y_test, axis=1)

# TODO: подобрать гиперпараметры отдельных моделей

# инициализирем модели с подобранными гиперпараметрами
estimators = [RandomForestClassifier(random_state=54232), 
              ExtraTreesClassifier(random_state=23412),
              AdaBoostClassifier(random_state=24212), 
              GradientBoostingClassifier(random_state=2732982)]

# получаем предсказания вероятностей ансамблей на кросс-валидации для обучающей выборки
predicted = [cross_val_predict_proba(est, X_train, y_train, X_test) for est in estimators]

X_train_stack = np.stack([p[0] for p in predicted], axis=1)
X_test_stack = np.stack([p[1] for p in predicted], axis=1)

## Объединяем предсказания ансамблей с помощью логистической регрессии

In [134]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

kfold = KFold(shuffle=True, n_splits=4, random_state=19746)
params = {'class_weight': ['balanced', None],
          'penalty': ['l1', 'l2'],
          'C': [0.4, 0.5, 1., 2., 2.5, 3., 3.5, 4.]}

grid = GridSearchCV(LogisticRegression(), params, scoring='neg_log_loss', cv=kfold)
grid.fit(X_train_stack, y_train)
pd.DataFrame(grid.cv_results_).sort_values('rank_test_score')[:3]

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_C,param_class_weight,param_penalty,params,rank_test_score,split0_test_score,...,split1_test_score,split1_train_score,split2_test_score,split2_train_score,split3_test_score,split3_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
10,0.002841,0.000840,-0.411206,-0.407174,1,None,l1,"{'C': 1.0, 'class_weight': None, 'penalty': 'l1'}",1,-0.473942,...,-0.423562,-0.402609,-0.385148,-0.416383,-0.361953,-0.424017,0.000617,0.000190,0.042398,0.014586
23,0.000990,0.000761,-0.411394,-0.407007,3,None,l2,"{'C': 3.0, 'class_weight': None, 'penalty': 'l2'}",2,-0.473668,...,-0.422555,-0.402864,-0.387398,-0.415697,-0.361733,-0.424300,0.000066,0.000038,0.041957,0.014737
19,0.001004,0.000851,-0.411409,-0.407207,2.5,None,l2,"{'C': 2.5, 'class_weight': None, 'penalty': 'l2'}",3,-0.472913,...,-0.422588,-0.403041,-0.387816,-0.415890,-0.362100,-0.424454,0.000099,0.000203,0.041511,0.014696


In [135]:
grid.best_estimator_.coef_

array([[ 0.69409073,  0.16450228,  0.        ,  4.30609215]])

In [136]:
grid.best_estimator_.fit(X_train_stack, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [137]:
grid.best_estimator_.coef_

array([[ 0.69672038,  0.16442023,  0.        ,  4.30346911]])

In [130]:
predicted = grid.best_estimator_.predict(X_test_stack)

## Формируем фалй для отправки

In [131]:
with open('submission.txt', 'w') as out:
    out.write('PassengerId,Survived\n')
    for passenger, y in zip(test_passenger_id, predicted):
        out.write('%s,%s\n' % (passenger, y))